In [6]:
!pip install torch torchvision

     |████████████████████████████████| 19.1 MB 16.1 MB/s eta 0:00:01


In [7]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import os
import shutil